In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
import os
import warnings

# 파일 불러와서 열이름 바꾸기
korean, long_term_frgn, short_term_frgn, resident, card_cnt, card_amt, waste_cnt
(이거 예시는 card_amt임)

In [2]:
df = pd.read_csv("files\\data_preprocessing\\long_term_frgn_preprocessing.csv", encoding='cp949')

# 시계열 피처단위로 변경
df['base_date'] = pd.to_datetime(df['base_date'])

# base_date를 index로 설정
df.index = df['base_date']
df.columns = ['base_date', 'emd_cd', 'long_term_frgn']  
df

,base_date,emd_cd,long_term_frgn
base_date,,,
2018-01-31,2018-01-31,50110250,427986.3363
2018-02-28,2018-02-28,50110250,427240.8119
2018-03-31,2018-03-31,50110250,513638.2631
2018-04-30,2018-04-30,50110250,474601.3510
2018-05-31,2018-05-31,50110250,395262.5483
...,...,...,...
2021-02-28,2021-02-28,50130620,109325.4866
2021-03-31,2021-03-31,50130620,118258.8891
2021-04-30,2021-04-30,50130620,129227.7480


In [3]:
# 필요 없는 열 제거
data = df.drop(['base_date'], 1)    
data

,emd_cd,long_term_frgn
base_date,,
2018-01-31,50110250,427986.3363
2018-02-28,50110250,427240.8119
2018-03-31,50110250,513638.2631
2018-04-30,50110250,474601.3510
2018-05-31,50110250,395262.5483
...,...,...
2021-02-28,50130620,109325.4866
2021-03-31,50130620,118258.8891
2021-04-30,50130620,129227.7480


In [4]:
# 한 행정동의 행만 추출
sub_area = data['emd_cd'] == 50110250 

# base_date, card_amt 만 갖는 df 생성
sub_df = data[sub_area]
sub_df = sub_df.drop('emd_cd', 1)
sub_df

,long_term_frgn
base_date,
2018-01-31,427986.3363
2018-02-28,427240.8119
2018-03-31,513638.2631
2018-04-30,474601.3510
2018-05-31,395262.5483
2018-06-30,442089.2799
2018-07-31,515032.9625
2018-08-31,630295.3224
2018-09-30,512443.3142


In [5]:
# series 형으로 변환
sub_df.to_dict('series')

{'long_term_frgn': base_date
 2018-01-31    427986.3363
 2018-02-28    427240.8119
 2018-03-31    513638.2631
 2018-04-30    474601.3510
 2018-05-31    395262.5483
 2018-06-30    442089.2799
 2018-07-31    515032.9625
 2018-08-31    630295.3224
 2018-09-30    512443.3142
 2018-10-31    464248.3419
 2018-11-30    407083.6888
 2018-12-31    543321.1395
 2019-01-31    638746.3328
 2019-02-28    567936.0292
 2019-03-31    710199.6022
 2019-04-30    559031.6529
 2019-05-31    586962.1745
 2019-06-30    506628.5975
 2019-07-31    574927.0618
 2019-08-31    640747.4311
 2019-09-30    564068.2223
 2019-10-31    555618.8033
 2019-11-30    469689.3527
 2019-12-31    482920.2385
 2020-01-31    587299.9640
 2020-02-29    559439.3959
 2020-03-31    714445.0812
 2020-04-30    638421.4188
 2020-05-31    528744.9598
 2020-06-30    615636.9501
 2020-07-31    544030.6248
 2020-08-31    491175.6471
 2020-09-30    518727.7307
 2020-10-31    477503.1113
 2020-11-30    355385.3711
 2020-12-31    381481.4364

# ARIMA

In [6]:
# ARIMA에 적합한 float로 바꾸는 과정
sub_df_float = sub_df[:].astype(np.float)

In [7]:
# ADF 검정 - 정상성 확인하는 검정 (귀무 : 정상성 만족 x, 대립 : 정상성 만족)
from statsmodels.tsa.stattools import adfuller

result = adfuller(sub_df)
print('ADF Statistic : %f' % result[0])
print('p-value : %f' % result[1])
print('Critical Values :')
for key, value in result[4].items():
    print('\t%s: %3f' % (key, value))

ADF Statistic : -3.174369
p-value : 0.021517
Critical Values :
	1%: -3.600983
	5%: -2.935135
	10%: -2.605963


In [8]:
from pmdarima.arima import auto_arima

model_arima= auto_arima(sub_df,trace=True, error_action='ignore',suppress_warnings=True,stepwise=False,seasonal=True)

model_arima.fit(sub_df)

 ARIMA(0,0,0)(0,0,0)[1] intercept   : AIC=1080.091, Time=0.02 sec
 ARIMA(0,0,1)(0,0,0)[1] intercept   : AIC=1070.712, Time=0.09 sec
 ARIMA(0,0,2)(0,0,0)[1] intercept   : AIC=1070.195, Time=0.04 sec
 ARIMA(0,0,3)(0,0,0)[1] intercept   : AIC=1068.221, Time=0.04 sec
 ARIMA(0,0,4)(0,0,0)[1] intercept   : AIC=1069.536, Time=0.08 sec
 ARIMA(0,0,5)(0,0,0)[1] intercept   : AIC=1070.442, Time=0.07 sec
 ARIMA(1,0,0)(0,0,0)[1] intercept   : AIC=1064.197, Time=0.02 sec
 ARIMA(1,0,1)(0,0,0)[1] intercept   : AIC=1065.971, Time=0.05 sec
 ARIMA(1,0,2)(0,0,0)[1] intercept   : AIC=1067.908, Time=0.05 sec
 ARIMA(1,0,3)(0,0,0)[1] intercept   : AIC=1070.024, Time=0.11 sec
 ARIMA(1,0,4)(0,0,0)[1] intercept   : AIC=1072.751, Time=0.13 sec
 ARIMA(2,0,0)(0,0,0)[1] intercept   : AIC=1065.915, Time=0.04 sec
 ARIMA(2,0,1)(0,0,0)[1] intercept   : AIC=1067.934, Time=0.11 sec
 ARIMA(2,0,2)(0,0,0)[1] intercept   : AIC=1068.129, Time=0.23 sec
 ARIMA(2,0,3)(0,0,0)[1] intercept   : AIC=inf, Time=0.34 sec
 ARIMA(3,0,0)(0

ARIMA(order=(1, 0, 0), scoring_args={}, seasonal_order=(0, 0, 0, 1),
      suppress_warnings=True)

In [9]:
# ARIMA 모델 
from statsmodels.tsa.arima_model import ARIMA
import statsmodels.api as sm

# (AR=2, 차분=1, MA=2) 파라미터로 ARIMA 모델을 학습합니다.
model = ARIMA(sub_df.long_term_frgn.values, order=(1,0,0))

#trend : constant를 가지고 있는지, c - constant / nc - no constant
#disp : 수렴 정보를 나타냄
model_fit = model.fit(trend='c', full_output=True, disp=True)
print(model_fit.summary())

                              ARMA Model Results                              
Dep. Variable:                      y   No. Observations:                   42
Model:                     ARMA(1, 0)   Log Likelihood                -529.116
Method:                       css-mle   S.D. of innovations          71249.615
Date:                Sat, 11 Sep 2021   AIC                           1064.233
Time:                        03:52:13   BIC                           1069.446
Sample:                             0   HQIC                          1066.144
                                                                              
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const       5.132e+05   2.61e+04     19.692      0.000    4.62e+05    5.64e+05
ar.L1.y        0.5915      0.124      4.789      0.000       0.349       0.834
                                    Roots           

C:\Users\soeun\anaconda3\lib\site-packages\statsmodels\tsa\arima_model.py:472: FutureWarning: 
statsmodels.tsa.arima_model.ARMA and statsmodels.tsa.arima_model.ARIMA have
been deprecated in favor of statsmodels.tsa.arima.model.ARIMA (note the .
between arima and model) and
statsmodels.tsa.SARIMAX. These will be removed after the 0.12 release.

statsmodels.tsa.arima.model.ARIMA makes use of the statespace framework and
is both well tested and maintained.

To silence this warning and continue using ARMA and ARIMA until they are
removed, use:

import warnings
warnings.filterwarnings('ignore', 'statsmodels.tsa.arima_model.ARMA',
                        FutureWarning)
warnings.filterwarnings('ignore', 'statsmodels.tsa.arima_model.ARIMA',
                        FutureWarning)

  warnings.warn(ARIMA_DEPRECATION_WARN, FutureWarning)


# 예측하기

In [10]:
# 2단위 이후의 예측결과
fore = model_fit.forecast(steps=2)
print(fore)

(array([469672.43494702, 487440.02623898]), array([71249.61522209, 82781.28421869]), array([[330025.75519938, 609319.11469466],
       [325191.69057638, 649688.36190157]]))
